## Названия переменных с итоговыми таблицами:
**result_tail** - таблица остатков для начальника

**df_tail** - таблица остатков для корректировки

**total_fin** - общая таблица финансирования по ТОВД

**fin_tovd** - таблица финансирования для конкретного ТОВД

**fin_oumts**- таблица финансирования по ОУМТСам

## Подготовка к написанию

**А. Вводится переменная-список *all_failes* для всех файлов-распределений;**

**В. Импортируем Библиотеку Панд и НаН из Нампая на будущее;**

**С. Вводится функция *cleaner*, обрабатывающая таблицу в нужный для кода формат:**

*С.1.* Автоматически проставляю соответствующие задания ГОЗ в каждой строке;

*С.2.* Автоматически проставляю соответствующие КБК в каждой строке;

*С.3.* Превращаю КБК в object.

*Добавить срез по КБК, если понадобится*

In [1]:
import pandas as pd
from numpy import nan
all_failes = [] #переменная для всех файлов распределений
def cleaner(tabl):
    tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].astype('object')
    return tabl

## 1. Открываем все таблицы в переменные *(НАПИСАТЬ ЦИКЛ ДЛЯ ПАПКИ)*

In [2]:
#Временная проба пера на примере пары таблиц
df1 = pd.read_excel('Test1.xlsx')
df2 = pd.read_excel('Test2.xlsx')

## 2. Обработка данных

*Чистим ранее введенной функцией.*

In [3]:
df1 = cleaner(df1)
df2 = cleaner(df2)

### 3. Объединение всех распределений

*Происходит по столбцам с заданиями ГОЗ, наименованиям закупки и КБК.* **(Нужно удалить в этом окне переменную frames. Она определена еще в первом действии)**

In [4]:
all_failes = [df1, df2]
cat = pd.concat(all_failes)

## 4. Группировка объединенных таблиц

*Практически готовая таблица по финансированию тер органов в переменной df_pivo.*

**4.1** Осущестляется группировка по заданиям ГОЗ, КБК и наименованиям закупки в переменно дф_пиво;

**4.2** Апгрейд дф_пива и добавление столбца "распределение/отзыв" с суммой распределений и отзывов по всем ТОВД.

In [5]:
df_pivo = cat.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=cat.columns[3:], aggfunc='sum', sort=False)
#Это pivo для Вани
df_pivo.loc[:, 'Распределение/отзыв'] = 0
tovd = df_pivo.columns[:-1]
df_pivo['Распределение/отзыв'] = df_pivo[tovd].agg('sum', axis="columns")



## 5. Первые итоговые таблицы *ОСТАТКОВ*.

**5.1** Создание промежуточной таблицы с движением ЛБО по наименованиям закупки и КБК в переменной df_traffic со столбцами ЛБО и Кол-во, группируем;

**5.2** Открываем предварительно созданную таблицу по запланированным/откорректированным ЛБО в переменной lbo и обрабатываем функцией клинера. Группируем по заданиям ГОЗ, наименованиям закупки и КБК;

**5.3** **Создание итоговой таблицы №1 для корректировок в переменной *df_tail*:**

    5.3.1 Объединяем таблицу трафика и ЛБО;
    5.3.2 Создаем функцию *tail_calc* для подсчета остатков (ЛБО-Распределения/отзывы);
    5.3.3 Создаем столбик остатков и применяем к нему функцию *tail_calc*;
    5.3.4 Создаем столбик с остатками исключительно по заданиям ГОЗ (без наименования закупки), заполняем его через цикл построчно.
**5.4** **Создаем вторую итоговую таблицу №2 для начальства в переменной *result_tail*:**

    5.4.1 Удаляем ненужные столбцы;
    5.4.2 Обнуляем группировку через reset_index;
    5.4.3 Группируем по-новому, как надо и переименовываем стобцы.

In [6]:
#1
df_traffic = df_pivo['Распределение/отзыв'].to_frame(name='Распределение/отзыв')
df_traffic.loc[:, "Кол-во"] = 0
df_traffic.loc[:, "ЛБО"] = 0
#2
lbo = pd.read_excel('Лимиты 2023 211.xlsx')
lbo = cleaner(lbo)
lbo = lbo.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=lbo.columns[3:], sort=False)
#3.1
df_tail = pd.concat([df_traffic, lbo])
df_tail = df_tail.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=['Распределение/отзыв', 'Кол-во', 'ЛБО'], aggfunc='sum', sort=False)

#3.2
# Функция для подсчета остатков:
def tail_calc (row):
    return row['ЛБО']-row['Распределение/отзыв'] 

#3.3
df_tail['Остатки'] = df_tail.apply(tail_calc, axis=1)
df_tail

#3.4
df_tail.loc[:, 'Сгруппированные остатки'] = nan
for group in df_tail.index.get_level_values(0).unique():
    namepurch = df_tail.loc[[group]].index[0][1]
    kbk = df_tail.loc[[group]].index[0][2]
    df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()

#4.1
result_tail = df_tail.droplevel(level=1)
del result_tail['Кол-во']
del result_tail['ЛБО']
del result_tail ['Распределение/отзыв']

#4.2
result_tail = result_tail.reset_index()

#4.3
result_tail = result_tail.groupby(['Задание ГОЗ', 'КБК'], sort=False) ['Сгруппированные остатки', 'Остатки'].sum()
result_tail = result_tail.rename (
columns={
    'Сгруппированные остатки':'Остатки по заданию ГОЗ',
    'Остатки':'Остатки по КБК',
}
)

C:\Users\dtrusheva\AppData\Local\Temp\ipykernel_6020\902448765.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\dtrusheva\AppData\Local\Temp\ipykernel_6020\902448765.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\dtrusheva\AppData\Local\Temp\ipykernel_6020\902448765.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\dtrusheva\AppData\Local\Temp\ipykernel_6020\902448765.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users